In [4]:
#1.Importing libraries and dependencies :

#Importing Qiskit
import qiskit

In [2]:
#Importing everything from qiskit
from qiskit import *

In [ ]:
# Import TensorFlow and TensorFlow Quantum
import tensorflow as tf
import tensorflow_quantum as tfq

# Import Cirq for quantum circuit creation and simulation
import cirq

# Import NumPy and Matplotlib for data manipulation and visualization
import numpy as np
import matplotlib.pyplot as plt

# Import OpenCV, PIL, and scikit-image for image processing
import cv2
from PIL import Image
import skimage

# Import os and glob for file handling
import os
import glob

# Import tqdm for progress bar
from tqdm import tqdm


In [ ]:
#2.Loading and preprocessing the dataset:


# Define the paths to the dataset folders

original_images_path = "1.Original Images"
groundtruths_path = "2.All Segments Groundtruths"

# Define the subfolders for each segment

segments = ["1. Microaneurysms", "2.Hemorrhages", "3. Hard Exudates", "4.Soft Exudates", "5.Optic Disc"]

# Define the image size and number of channels

img_size = 256
img_channels = 3

# Define a function to load and preprocess an image

def load_and_preprocess_image(image_path):
  # Read the image as a numpy array
  image = cv2.imread(image_path)
  # Resize the image to the desired size
  image = cv2.resize(image, (img_size, img_size))
  # Convert the image from BGR to RGB color space
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  # Normalize the image to have values between 0 and 1
  image = image / 255.0
  # Return the image
  return image

# Define a function to load and preprocess a groundtruth

def load_and_preprocess_groundtruth(groundtruth_path):
  # Read the groundtruth as a numpy array

  groundtruth = cv2.imread(groundtruth_path, cv2.IMREAD_GRAYSCALE)
  # Resize the groundtruth to the desired size

  groundtruth = cv2.resize(groundtruth, (img_size, img_size))
  # Threshold the groundtruth to have binary values

  _, groundtruth = cv2.threshold(groundtruth, 127, 255, cv2.THRESH_BINARY)
  # Normalize the groundtruth to have values between 0 and 1

  groundtruth = groundtruth / 255.0
  # Return the groundtruth

  return groundtruth

# Define empty lists to store the images and groundtruths

images = []
groundtruths = []

# Loop over each segment subfolder
for segment in segments:
  # Get the list of image files in the original images folder

  image_files = glob.glob(os.path.join(original_images_path, "*.jpg"))
  # Get the list of groundtruth files in the corresponding segment folder

  groundtruth_files = glob.glob(os.path.join(groundtruths_path, segment, "*.tif"))
  
  # Loop over each image file and its corresponding groundtruth file
    
  for image_file, groundtruth_file in zip(image_files, groundtruth_files):
    # Load and preprocess the image and the groundtruth
    
    image = load_and_preprocess_image(image_file)
    groundtruth = load_and_preprocess_groundtruth(groundtruth_file)
    # Append them to the lists
    
    images.append(image)
    groundtruths.append(groundtruth)

# Convert the lists to numpy arrays

images = np.array(images)
groundtruths = np.array(groundtruths)

# Print the shapes of the arrays

print("Images shape:", images.shape)
print("Groundtruths shape:", groundtruths.shape)


In [ ]:
#3.Encoding the images into quantum data :

# Define the number of qubits to use for encoding

n_qubits = 5

# Define a function to encode an image into a quantum state using amplitude encoding

def amplitude_encoding(image):
  # Reshape the image into a 1D array

  image = tf.reshape(image, [img_size * img_size * img_channels])
  # Normalize the image to have unit norm

  image = image / tf.norm(image)
    
  # Create a circuit

  circuit = cirq.Circuit()
    
  # Add a state preparation gate for each qubit

  for i in range(n_qubits):
    circuit.append(cirq.ry(np.arccos(image[i])).on(cirq.GridQubit(0, i)))
  return circuit

# Define a function to encode a batch of images into quantum data using amplitude encoding

def encode_batch(images):
    
  # Create an empty list to store the circuits

  circuits = []
    
  # Loop over each image in the batch

  for image in images:
        
    # Encode the image into a circuit
    
    circuit = amplitude_encoding(image)
    
    # Append the circuit to the list
    
    circuits.append(circuit)
    
  # Convert the list of circuits to a tensor
  circuits = tfq.convert_to_tensor(circuits)
    
  # Return the tensor of circuits
  return circuits

# Encode the images and groundtruths into quantum data using amplitude encoding
images_quantum = encode_batch(images)
groundtruths_quantum = encode_batch(groundtruths)

# Print the shapes of the quantum data tensors
print("Images quantum shape:", images_quantum.shape)
print("Groundtruths quantum shape:", groundtruths_quantum.shape)


In [ ]:
#4.Creating quantum convolution filters :


# Define the number of filters to use
n_filters = 4

# Define a function to create a parameterized quantum circuit that acts as a convolution filter
def create_filter_circuit():
    
  # Create a circuit
  circuit = cirq.Circuit()
    
  # Add a layer of Hadamard gates to create superposition
  for i in range(n_qubits):
    circuit.append(cirq.H(cirq.GridQubit(0, i)))
    
  # Add a layer of parameterized rotation gates to create entanglement and interference

  for i in range(n_qubits):
    circuit.append(cirq.rx(tf.Variable(np.random.uniform(0, 2 * np.pi))).on(cirq.GridQubit(0, i)))
    
  # Return the circuit
  return circuit

# Define a function to create a measurement operator that defines the output of the filter

def create_output_operator():
    
  # Create an empty list to store the measurement operators

  operators = []
    
  # Loop over each filter

  for i in range(n_filters):
        
    # Create a random subset of qubits to measure
    
    qubits = np.random.choice(range(n_qubits), size=int(n_qubits / 2), replace=False)
    
    # Create a Pauli sum of Z operators on the selected qubits
    
    operator = sum([cirq.Z(cirq.GridQubit(0, j)) for j in qubits])
    
    # Append the operator to the list
    
    operators.append(operator)
    
  # Return the list of operators

  return operators



# Create a list of filter circuits using the create_filter_circuit function

filter_circuits = [create_filter_circuit() for _ in range(n_filters)]

# Create a list of output operators using the create_output_operator function
output_operators = create_output_operator()



# Print the filter circuits and output operators
for i in range(n_filters):
  print(f"Filter {i+1} circuit:")
  print(filter_circuits[i])
  print(f"Filter {i+1} output operator:")
  print(output_operators[i])


In [ ]:
#5.Building and training the model :

# Define the batch size and number of epochs
batch_size = 32
epochs = 10

# Define a function to create a quantum convolutional layer
def create_qcnn_layer():
  # Create a sequential layer
  layer = tf.keras.Sequential([
    # Add a quantum convolution filter
    tfq.layers.PQC(
      # Use one of the filter circuits from the list
      filter_circuits[np.random.randint(n_filters)],
      # Use one of the output operators from the list
      output_operators[np.random.randint(n_filters)]
    ),
    # Add a batch normalization layer
    tf.keras.layers.BatchNormalization(),
    # Add a ReLU activation layer
    tf.keras.layers.ReLU()
  ])
  # Return the layer
  return layer

# Define a function to create a classical dense layer
def create_dense_layer(units):
  # Create a sequential layer
  layer = tf.keras.Sequential([
    # Add a dense layer with the given number of units
    tf.keras.layers.Dense(units),
    # Add a batch normalization layer
    tf.keras.layers.BatchNormalization(),
    # Add a ReLU activation layer
    tf.keras.layers.ReLU()
  ])
  # Return the layer
  return layer

# Build the quantum convolutional neural network model
model = tf.keras.Sequential([
  # Add an amplitude encoding circuit to encode the images into quantum data
  tfq.layers.AddCircuit(
    lambda x: tfq.convert_to_tensor([amplitude_encoding(x)])
  ),
  # Add two quantum convolutional layers
  create_qcnn_layer(),
  create_qcnn_layer(),
  # Add a flatten layer to convert the quantum data into classical data
  tfq.layers.Flatten(),
  # Add two classical dense layers for classification
  create_dense_layer(64),
  create_dense_layer(32),
  # Add a final dense layer with sigmoid activation for binary output
  tf.keras.layers.Dense(1, activation="sigmoid")
])

# Compile the model with binary crossentropy loss and Adam optimizer
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Print the model summary
model.summary()

# Train the model on the images and groundtruths using fit method
model.fit(images_quantum, groundtruths_quantum, batch_size=batch_size, epochs=epochs, validation_split=0.2)


In [ ]:
#6.Evaluating and visualizing the results :

# Define a function to plot an image and its groundtruth
def plot_image_and_groundtruth(image, groundtruth):
  # Create a figure with two subplots
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
  # Plot the image on the first subplot
  ax1.imshow(image)
  ax1.set_title("Image")
  ax1.axis("off")
  # Plot the groundtruth on the second subplot
  ax2.imshow(groundtruth, cmap="gray")
  ax2.set_title("Groundtruth")
  ax2.axis("off")
  # Show the figure
  plt.show()

# Define a function to plot an image and its prediction
def plot_image_and_prediction(image, prediction):
  # Create a figure with two subplots
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
  # Plot the image on the first subplot
  ax1.imshow(image)
  ax1.set_title("Image")
  ax1.axis("off")
  # Plot the prediction on the second subplot
  ax2.imshow(prediction, cmap="gray")
  ax2.set_title("Prediction")
  ax2.axis("off")
  # Show the figure
  plt.show()

# Define a function to calculate and plot the ROC curve and AUC score
def plot_roc_curve(y_true, y_pred):
  # Calculate the false positive rate and true positive rate
  fpr, tpr, _ = tf.keras.metrics.roc_curve(y_true, y_pred)
  # Calculate the area under the curve
  auc = tf.keras.metrics.auc(fpr, tpr)
  # Create a figure
  fig = plt.figure(figsize=(5, 5))
  # Plot the ROC curve
  plt.plot(fpr, tpr, label=f"AUC = {auc:.3f}")
  # Plot the diagonal line
  plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
  # Set the title and labels
  plt.title("ROC Curve")
  plt.xlabel("False Positive Rate")
  plt.ylabel("True Positive Rate")
  plt.legend()
  # Show the figure
  plt.show()

# Evaluate the model on the test set using predict method
predictions = model.predict(images_quantum[800:])

# Reshape the predictions and groundtruths to have the same shape as the images
predictions = predictions.reshape(-1, img_size, img_size)
groundtruths = groundtruths.reshape(-1, img_size, img_size)

# Plot some examples of images and their groundtruths and predictions
for i in range(5):
    plot_image_and_groundtruth(images[i + 800], groundtruths[i + 800])
    plot_image_and_prediction(images[i + 800], predictions[i + 800])

# Plot the ROC curve and AUC score for the test set
plot_roc_curve(groundtruths[800:], predictions)
